Practical 8

Write a program to implement RNN on sun-spot dataset for different time steps as 5, 8, 12, 15 and compare the results for training & testing error. Also, find training and testing accuracy with LSTM.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM

# Load the dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-sunspots.csv"
df = pd.read_csv(url, usecols=[0,1], skiprows=1, names=['Month', 'Sunspots'], index_col=0)

# Normalize the data
scaler = MinMaxScaler (feature_range=(0, 1))
data = scaler.fit_transform(df)

# Define the function to create a windowed dataset

def create_dataset(data, window_size):
  X, Y = [], []
  for i in range(len(data)-window_size):
    X.append(data[i:i+window_size])
    Y.append(data[i+window_size])
  return np.array(X), np.array(Y)

# Set the window sizes to test
window_sizes = [5, 8, 12, 15]

# Split the dataset into training and testing sets
split_fraction = 0.8
split_index = int(split_fraction * len(data))
train_data = data[:split_index]
test_data = data[split_index:]

timeStep_dict = { 5 : [] , 8: [], 12: [] , 15: [] }
# Train and evaluate the models for each window size
for window_size in window_sizes:

  # Create the windowed dataset
  X_train, Y_train = create_dataset (train_data, window_size)
  X_test, Y_test = create_dataset (test_data, window_size)

  # Define the RNN model
  rnn_model = Sequential()
  rnn_model.add(SimpleRNN (units=3, activation='relu', input_shape=(window_size, 1)))
  rnn_model.add (Dense (units=1))
  rnn_model.compile(optimizer='adam', loss='mse')
  rnn_model.summary()

  # Train the RNN model
  rnn_history = rnn_model.fit(X_train, Y_train, epochs=10, batch_size=16, validation_split=0.2)

  # Define the LSTM model
  lstm_model = Sequential()
  lstm_model.add(LSTM(units=64, activation='relu', input_shape=(window_size, 1)))
  lstm_model.add(Dense (units=1))
  lstm_model.compile(optimizer='adam', loss='mse', metrics = ['accuracy'])
  lstm_model.summary()
  # Train the LSTM model
  lstm_history = lstm_model.fit(X_train, Y_train, epochs=10, batch_size=16, validation_split =0.2)

  # Evaluate the models
  rnn_train_error = rnn_model.evaluate(X_train, Y_train, verbose=0)
  rnn_test_error = rnn_model.evaluate(X_test, Y_test, verbose=0)
  lstm_train_acc = lstm_model.evaluate(X_train, Y_train, verbose=0)[1]*100
  lstm_test_acc = lstm_model.evaluate(X_test, Y_test, verbose=0)[1]*100

  timeStep_dict[window_size].append([rnn_train_error, rnn_test_error, lstm_train_acc, lstm_test_acc])

  print("Window size:", window_size)
  print("RNN training error:", rnn_train_error)
  print("RNN testing error:", rnn_test_error)
  print("LSTM training accuracy:", lstm_train_acc)
  print("LSTM testing accuracy:", lstm_test_acc)




  print()


Model: "sequential_32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_17 (SimpleRNN)   (None, 3)                 15        
                                                                 
 dense_31 (Dense)            (None, 1)                 4         
                                                                 
Total params: 19
Trainable params: 19
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
113/113 [==============================] - 2s 6ms/step - loss: 0.0640 - val_loss: 0.0192
Epoch 2/10
113/113 [==============================] - 0s 4ms/step - loss: 0.0293 - val_loss: 0.0097
Epoch 3/10
113/113 [==============================] - 0s 3ms/step - loss: 0.0182 - val_loss: 0.0063
Epoch 4/10
113/113 [==============================] - 0s 3ms/step - loss: 0.0130 - val_loss: 0.0049
Epoch 5/10
113/113 [==============================] -

In [ ]:
# storing the values as a dict
import pandas as pd
compareModels = {'Time Step': ['5', '8', '12', '15'],
                 'RNN-training loss' : [timeStep_dict[5][0][0],
                                        timeStep_dict[8][0][0],
                                        timeStep_dict[12][0][0],
                                        timeStep_dict[15][0][0]],
                'RNN-testing loss' : [timeStep_dict[5][0][1],
                                        timeStep_dict[8][0][1],
                                        timeStep_dict[12][0][1],
                                        timeStep_dict[15][0][1]],
                'LSTM-training accuracy' : [timeStep_dict[5][0][2],
                                        timeStep_dict[8][0][2],
                                        timeStep_dict[12][0][2],
                                        timeStep_dict[15][0][2]],
                'LSTM-testing accuracy' : [timeStep_dict[5][0][3],
                                        timeStep_dict[8][0][3],
                                        timeStep_dict[12][0][3],
                                        timeStep_dict[15][0][3]]}
table = pd.DataFrame(compareModels)
table

,Time Step,RNN-training loss,RNN-testing loss,LSTM-training accuracy,LSTM-testing accuracy
0,5,0.005427,0.018627,2.976455,0.178891
1,8,0.021586,0.061488,2.980427,0.179856
2,12,0.021573,0.060911,2.985740,0.181159
3,15,0.003877,0.006814,2.989737,0.182149
